# Baidu Emotion

In [1]:
#Adjust the working folder
import sys
import os
#print(globals())
file_folder = globals()['_dh'][0]
wk_dir = os.path.dirname(file_folder)
os.chdir(wk_dir)

import pandas as pd
# from code_felix.utils_.util_pandas import *
from code_felix.core.config import *
# from code_felix.feature.category import *
import matplotlib.pyplot as plt
# from code_felix.feature.read_file import _get_transaction, _summary_card_trans_col

#trans_new =  get_trans(trans_new_file)

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

2018-12-31 22:47:41,491 util_log.py[61] DEBUG Start the program at:LALI2-M-G0MD, 127.0.0.1, with:Load module


In [ ]:
# https://www.kaggle.com/eray1yildiz/using-lstms-with-attention-for-emotion-recognition/notebook
# https://ai.tencent.com/ailab/nlp/embedding.html
#blog : https://blog.csdn.net/sinat_26917383/article/details/83999966

#如何利用词库,增强jieba: https://blog.csdn.net/chinatelecom08/article/details/84588071

In [6]:


import jieba
jieba.load_userdict(jieba_dict)

#text = "欧阳建国是创新办主任也是欢聚时代公司云计算方面的专家"
text = "真的挺好用, 吃不上"

# jieba.cut() 方法接受两个输入参数: 
# 需要分词的字符串
# cut_all 参数用来控制是否采用全模式

# 精确模式，默认模式就是精确模式
seg_list = jieba.cut(text, cut_all = False)
print('Default Mode:\n' + '/' .join(seg_list))

# 全模式
seg_list = jieba.cut(text, cut_all = True)
print( "Full Mode:\n" + '/' .join(seg_list))

# jieba.cut_for_search() 方法接受一个参数：
# 需要分词的字符串
# 该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细

# 搜索引擎模式
seg_list = jieba.cut_for_search(text)
print('Research Mode:\n' + '/'.join(seg_list))
 

Default Mode:
真的/挺好用/,/ /吃不上
Full Mode:
真的/挺好/挺好用////吃不上
Research Mode:
真的/挺好/挺好用/,/ /吃不上


In [7]:
jieba.cut?

In [ ]:
import jieba


print('Begin success')
jieba.load_userdict('./input/jieba.txt')

print('Load success')

train= pd.read_csv(train_file, encoding='gb18030', delimiter='\t', header=None)
train = train[:20]
train.head()
train['jieba'] = train.iloc[:,2].apply(lambda text: ' '.join(jieba.cut(str(text), cut_all = False)))
train['jieba_len'] = train['jieba'].apply(lambda text: len(text.split(' ')))
train.head()

import collections
count = collections.Counter()

for text in train['jieba'].values:
    for word in text.split(' '):
        count[word] +=1

print(len(count))
print(count)

In [ ]:
jieba.load_userdict('./input/jieba.txt')
dir(jieba.FREQ)

In [ ]:
list(jieba.dt.FREQ.keys())[:10]

In [5]:

import numpy as np
import gensim
from code_felix.core.config import *

def compute_ngrams(word, min_n, max_n):
    #BOW, EOW = ('<', '>')  # Used by FastText to attach to all words as prefix and suffix
    extended_word =  word
    ngrams = []
    for ngram_length in range(min_n, min(len(extended_word), max_n) + 1):
        for i in range(0, len(extended_word) - ngram_length + 1):
            ngrams.append(extended_word[i:i + ngram_length])
    return list(set(ngrams))


def wordVec(word,wv_from_text,min_n = 1, max_n = 3):
    '''
    ngrams_single/ngrams_more,主要是为了当出现oov的情况下,最好先不考虑单字词向量
    '''
    # 确认词向量维度
    word_size = wv_from_text.wv.syn0[0].shape[0]   
    # 计算word的ngrams词组
    ngrams = compute_ngrams(word,min_n = min_n, max_n = max_n)
    # 如果在词典之中，直接返回词向量
    if word in wv_from_text.wv.vocab.keys():
        return wv_from_text[word]
    else:  
        # 不在词典的情况下
        word_vec = np.zeros(word_size, dtype=np.float32)
        ngrams_found = 0
        ngrams_single = [ng for ng in ngrams if len(ng) == 1]
        ngrams_more = [ng for ng in ngrams if len(ng) > 1]
        # 先只接受2个单词长度以上的词向量
        for ngram in ngrams_more:
            if ngram in wv_from_text.wv.vocab.keys():
                word_vec += wv_from_text[ngram]
                ngrams_found += 1
                #print(ngram)
        # 如果，没有匹配到，那么最后是考虑单个词向量
        if ngrams_found == 0:
            for ngram in ngrams_single:
                word_vec += wv_from_text[ngram]
                ngrams_found += 1
        if word_vec.any():
            return word_vec / max(1, ngrams_found)
        else:
            raise KeyError('all ngrams for word %s absent from model' % word)
 



def load_embedding(path):
    embedding_index = {}
    f = open(path,encoding='utf8')
    for index,line in enumerate(f):
        if index == 0:
            continue
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray(values[1:],dtype='float32')
        embedding_index[word] = coefs
    f.close()

    return embedding_index




import jieba
def gen_mini_embedding(full, mini, file, cut_all=True):   

    input_text= pd.read_csv(file, encoding='gb18030', delimiter='\t', header=None)
    input_text = input_text[:200]
    input_text.head()
    input_text['jieba'] = input_text.iloc[:,2].apply(lambda text: ' '.join(jieba.cut(str(text), cut_all)))
    input_text['jieba_len'] = input_text['jieba'].apply(lambda text: len(text.split(' ')))
    for text in input_text['jieba'].values:
        for word in text.strip().split(' '):
            if word in full and word not in mini:
                mini[word] = full[word]
            elif word not in full and len(word) == 1:
                #print(f'Canot find vec for:{word}')
                pass
            elif word not in full and len(word) >1:
                print(f'Canot find vec for:{len(word)}, {word}')
            else:
                pass
    return mini


#compute_ngrams('搞来搞去', 1, 3)
#wordVec('搞来搞去', embed, 1, 3)   



embed = load_embedding(word2vec_model)
vector_size=200
mini = gensim.models.keyedvectors.Word2VecKeyedVectors(vector_size)
mini = gen_mini_embedding(embed, mini, train_file )

mini = gen_mini_embedding(embed, mini, test_file )

from gensim.test.utils import get_tmpfile
fname = "./output/mini.kv"
mini.save(fname)
            

Canot find vec for:2, 三方
Canot find vec for:2, 方平
Canot find vec for:4, 方便管理
Canot find vec for:2, 烤鸭
Canot find vec for:2, 标明
Canot find vec for:2, 越来
Canot find vec for:2, 作过
Canot find vec for:3, 叫外卖
Canot find vec for:2, 辣味
Canot find vec for:3, 叫外卖
Canot find vec for:2, 送达
Canot find vec for:2, 分毫
Canot find vec for:4, 分毫不差
Canot find vec for:6, hellip
Canot find vec for:6, hellip
Canot find vec for:2, 啊哈
Canot find vec for:4, 品种齐全
Canot find vec for:2, 上菜
Canot find vec for:4, 服务态度
Canot find vec for:2, 单系
Canot find vec for:3, 多一些
Canot find vec for:2, 定点
Canot find vec for:2, 上菜
Canot find vec for:3, 没得说
Canot find vec for:2, 刺身
Canot find vec for:2, 常新
Canot find vec for:2, 有求
Canot find vec for:4, 有求必应
Canot find vec for:2, 必应
Canot find vec for:2, 越来
Canot find vec for:4, 努力提高
Canot find vec for:2, 扇贝
Canot find vec for:2, 收银
Canot find vec for:2, 上菜
Canot find vec for:2, 色香
Canot find vec for:3, 色香味
Canot find vec for:2, 俱全
Canot find vec for:2, 里加
Canot find vec for:2, 青豆


Canot find vec for:3, 购物车
Canot find vec for:2, 不支
Canot find vec for:4, 饿着肚子
Canot find vec for:3, 速度慢
Canot find vec for:3, 不多见
Canot find vec for:2, 能感
Canot find vec for:2, 菜店
Canot find vec for:3, 吃不完
Canot find vec for:2, 五分
Canot find vec for:3, 整体感
Canot find vec for:2, 中餐
Canot find vec for:6, hellip
Canot find vec for:6, hellip
Canot find vec for:4, 服务周到
Canot find vec for:2, 周到
Canot find vec for:2, 香辣
Canot find vec for:2, 清江
Canot find vec for:3, 挺好吃
Canot find vec for:2, 小块
Canot find vec for:3, 优惠券
Canot find vec for:4, 功能强大
Canot find vec for:2, 咖喱
Canot find vec for:3, 吃不完
Canot find vec for:2, 少点
Canot find vec for:2, 点菜
Canot find vec for:2, 少点
Canot find vec for:4, 重庆火锅
Canot find vec for:3, 大鸭梨
Canot find vec for:2, 鸭梨
Canot find vec for:2, 种菜
Canot find vec for:2, 菜色
Canot find vec for:2, 越来
Canot find vec for:2, 糊弄
Canot find vec for:2, 作到
Canot find vec for:2, 动工
Canot find vec for:2, 面馆
Canot find vec for:2, 婚宴
Canot find vec for:2, 赶在
Canot find vec for:2, 上菜


Canot find vec for:2, 肯德
Canot find vec for:2, 九鼎
Canot find vec for:2, 配送
Canot find vec for:3, 承包商
Canot find vec for:2, 包商
Canot find vec for:2, 三轮
Canot find vec for:2, 送菜
Canot find vec for:2, 越来
Canot find vec for:2, 环球
Canot find vec for:2, 太慢
Canot find vec for:2, 上菜
Canot find vec for:2, 哈根
Canot find vec for:4, 哈根达斯
Canot find vec for:2, 达斯
Canot find vec for:3, 总体性
Canot find vec for:2, 体性
Canot find vec for:2, 既往
Canot find vec for:2, 各样
Canot find vec for:2, 派送
Canot find vec for:2, 派送
Canot find vec for:3, 老字号
Canot find vec for:2, 字号
Canot find vec for:2, 既往
Canot find vec for:4, 服务到位
Canot find vec for:3, 吃不起
Canot find vec for:3, 转基因
Canot find vec for:3, 大豆油
Canot find vec for:2, 豆油
Canot find vec for:2, 加林
Canot find vec for:2, 出点
Canot find vec for:3, 价比高
Canot find vec for:2, 比高
Canot find vec for:4, 比较丰富
Canot find vec for:4, 品种齐全
Canot find vec for:4, 服务态度
Canot find vec for:3, 一个多
Canot find vec for:4, 一个多月
Canot find vec for:2, 实是
Canot find vec for:3, SEO
Cano

Canot find vec for:2, 素菜
Canot find vec for:2, 没味
Canot find vec for:2, 加料
Canot find vec for:2, 越来
Canot find vec for:2, 不值
Canot find vec for:3, 自助餐
Canot find vec for:3, 信息量
Canot find vec for:2, 小礼
Canot find vec for:2, 不吝
Canot find vec for:2, 吝惜
Canot find vec for:4, 再接再励
Canot find vec for:2, 造福
Canot find vec for:2, 香草
Canot find vec for:2, 香草
Canot find vec for:2, 原生
Canot find vec for:4, 宫爆鸡丁
Canot find vec for:2, 鸡丁
Canot find vec for:2, 蓝莓
Canot find vec for:2, 鸭蛋
Canot find vec for:2, 热菜
Canot find vec for:4, 一抢而空
Canot find vec for:3, 越做越
Canot find vec for:6, middot
Canot find vec for:6, middot
Canot find vec for:6, middot
Canot find vec for:6, middot
Canot find vec for:2, 没剩
Canot find vec for:3, 速度慢
Canot find vec for:2, 改改
Canot find vec for:2, 避避
Canot find vec for:2, 避暑
Canot find vec for:2, 调给
Canot find vec for:2, 蒸笼
Canot find vec for:3, 自助餐
Canot find vec for:2, 84
Canot find vec for:2, 份量
Canot find vec for:2, 烤鸭
Canot find vec for:3, 中国化
Canot find vec for:2, 

2018-12-31 20:37:23,592 utils.py[531] INFO saving Word2VecKeyedVectors object under ./output/mini.kv, separately None
2018-12-31 20:37:23,601 utils.py[635] INFO not storing attribute vectors_norm
2018-12-31 20:37:23,604 smart_open_lib.py[176] DEBUG {'kw': {}, 'mode': 'wb', 'uri': './output/mini.kv'}
2018-12-31 20:37:23,677 utils.py[544] INFO saved ./output/mini.kv


Canot find vec for:2, 实是
Canot find vec for:2, 必胜
Canot find vec for:3, 必胜客
Canot find vec for:2, 必胜
Canot find vec for:3, 必胜客
Canot find vec for:2, 快餐
Canot find vec for:2, 搞活
Canot find vec for:2, 订餐
Canot find vec for:2, 上菜
Canot find vec for:4, 热情周到
Canot find vec for:2, 周到
Canot find vec for:3, APP
Canot find vec for:3, 叫外卖
Canot find vec for:2, 单系
Canot find vec for:4, 服务态度
Canot find vec for:2, 上菜
Canot find vec for:2, 挺快
Canot find vec for:3, 太好了
Canot find vec for:2, 肯德
Canot find vec for:2, 炸鸡
Canot find vec for:4, 优惠活动
Canot find vec for:2, 原味
Canot find vec for:2, 送款
Canot find vec for:2, 烤鸡
Canot find vec for:2, 鸡翅
Canot find vec for:2, 用纸
Canot find vec for:2, 纸袋
Canot find vec for:2, 袋装
Canot find vec for:2, 纸袋
Canot find vec for:2, 鸡皮
Canot find vec for:2, 黏住
Canot find vec for:2, 纸袋


In [54]:
mini.vocab.keys()

dict_keys(['买', '这套', '系统', '本来', '是用', '用来', '来做', '我们', '公司', '的', '公众', '公众号', '第三', '第三方', '平台', '运营', '没想', '没想到', '想到', '还有', 'app', '而且', '每个', '都很', '很方便', '方便', '管理', '是我', '我找', '找了', '很久', '的一', '一套', '程序', '还是', '是不', '不错', '别的', '菜', '没什么', '什么', '特殊', '使用', '说明', '看不', '看不懂', '不懂', '不会', '会用', '很多', '操作', '详细', '越来越', '不好', '好了', '菜品', '也', '少了', '服务', '也不', '不及', '及时', '是在', '不知', '不知道', '知道', '吃什么', '什么好', '就来', '来了', '跟', '这家', '家公司', '合作', '过一', '一次', '功能', '很强', '强大', '运行', '流畅', '很不', '很不错', '点的', '的中', '但是', '平时', '店里', '吃的', '可能', '外卖', '比', '直接', '花的', '时间', '时间长', '出来', '了的', '缘故', '吧', '味道', '是一', '是一样的', '一样', '样的', '好', '下次', '卖点', '试试', '满意', '速度', '真的', '的是', '是个', '预计', '毫不', '不差', '好评', '的确', '专业', '用心', '出品', '方面', '给', '还不', '有机', '有机会', '机会', '还来', '哈哈', '哈哈哈', '还', '品种', '齐全', '很快', '也不错', '态度', '感觉', '不过', '卫生', '过年', '当', '加了', '这个', '就是', '就是我', '需要', '要的', '感谢', '感谢你', '你们', '解决', '我的', '问题', '有新', '办理', '我还', '还会', '会来', '麻烦', '们的', '总体', '海鲜', '

In [3]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load(fname, mmap='r')
word_vectors.similar_by_word('系统')

2018-12-31 17:55:28,151 utils.py[415] INFO loading Word2VecKeyedVectors object from ./output/mini.kv
2018-12-31 17:55:28,156 smart_open_lib.py[176] DEBUG {'kw': {}, 'mode': 'rb', 'uri': './output/mini.kv'}
2018-12-31 17:55:28,185 utils.py[484] INFO setting ignored attribute vectors_norm to None
2018-12-31 17:55:28,187 utils.py[421] INFO loaded ./output/mini.kv
2018-12-31 17:55:28,191 keyedvectors.py[1358] INFO precomputing L2-norms of word weight vectors
/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('功能', 0.7172309160232544),
 ('模块', 0.7166910171508789),
 ('设备', 0.6992197036743164),
 ('智能', 0.6782375574111938),
 ('软件', 0.6644343137741089),
 ('程序', 0.661666989326477),
 ('优化', 0.6489428281784058),
 ('技术', 0.6255435943603516),
 ('界面', 0.6241154670715332),
 ('自动', 0.6087337732315063)]

In [ ]:
gen_mini_embedding(embed, mini, test_file )
len(mini)

In [34]:
from gensim.models import KeyedVectors
KeyedVectors.load?

2018-12-31 17:26:20,475 textcleaner.py[37] INFO 'pattern' package not found; tag filters are not available for English


In [ ]:
from file_cache.utils.util_date import timed
@timed()
def test():
    import gensim
    
    wv_from_text = gensim.models.KeyedVectors.load_word2vec_format(word2vec_model,binary=False)
    print(len(wv_from_text.vocab))
    
test()
                                                               

In [ ]:

print(wv_from_text.vocab.keys() )

In [ ]:
dir(wv_from_text)

In [11]:
test= pd.read_csv(test_file, encoding='gb18030', delimiter='\t', header=None)


test.shape


(35157, 3)